In [113]:
import os
import json
import numpy as np
from keras.layers import Embedding, Reshape, Merge, Dropout, Dense
from keras.models import Sequential, load_model
from keras.models import model_from_json
from keras.regularizers import l2

In [2]:
train_data_path = '/home/csdai0324/hw5_data/train.csv'
test_data_path = '/home/csdai0324/hw5_data/test.csv'
answer_path = '/home/csdai0324/hw5_data/answer.txt'

In [3]:
def read_train_data(train_data_path):
    with open(train_data_path, 'r') as train_data:
        train_data.readline()
        users = []
        movies = []
        ratings = []
        for data in train_data:
            lt = data.strip().split(',')
            users.append(int(lt[1]))
            movies.append(int(lt[2]))
            ratings.append(int(lt[3]))
    return users, movies, ratings

def read_test_data(test_data_path):
    with open(test_data_path, 'r') as test_data:
        test_data.readline()
        users = []
        movies = []
        for data in test_data:
            lt = data.strip().split(',')
            users.append(int(lt[1]))
            movies.append(int(lt[2]))
    return users, movies

users, movies, ratings = read_train_data(train_data_path)

In [4]:
def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

In [5]:
print(max(users), max(movies))

6040 3952


In [91]:
def mf_model(user_num, movie_num, factors):
    u = Sequential()
    u.add(Embedding(user_num + 1, factors, input_length=1, W_regularizer=l2(1e-6)))
    u.add(Reshape((factors,)))
    m = Sequential()
    m.add(Embedding(movie_num + 1, factors, input_length=1, W_regularizer=l2(1e-6)))
    m.add(Reshape((factors,)))
    model = Sequential()
    model.add(Merge([u, m], mode='dot'))
    model.compile(loss='mse', optimizer='adamax')
    return model

In [92]:
model = mf_model(max(users), max(movies), 256)
model.fit([np.array(users), np.array(movies)], ratings, batch_size=512, nb_epoch=1, verbose=1)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(6041, 256, embeddings_regularizer=<keras.reg..., input_length=1)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(3953, 256, embeddings_regularizer=<keras.reg..., input_length=1)`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
899873/899873 [==============================] - 4s - loss: 8.7677     


In [93]:
model.fit([np.array(users), np.array(movies)], ratings, batch_size=256, nb_epoch=5, verbose=1)

Epoch 1/5
 16896/899873 [..............................] - ETA: 8s - loss: 2.2642

/usr/local/lib/python3.5/dist-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


899873/899873 [==============================] - 8s - loss: 1.4258     
Epoch 2/5
899873/899873 [==============================] - 8s - loss: 0.9577     
Epoch 3/5
899873/899873 [==============================] - 8s - loss: 0.8650     
Epoch 4/5
899873/899873 [==============================] - 8s - loss: 0.8197     
Epoch 5/5
899873/899873 [==============================] - 8s - loss: 0.7853     


In [109]:
model.fit([np.array(users), np.array(movies)], ratings, batch_size=256, nb_epoch=1, verbose=1)

Epoch 1/1
 11008/899873 [..............................] - ETA: 10s - loss: 0.5264

/usr/local/lib/python3.5/dist-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


899873/899873 [==============================] - 8s - loss: 0.5177     


In [96]:
test_users, test_movies = read_test_data(test_data_path)

In [110]:
def predict_rating(userid, movieid):
    return model.predict([np.array([userid]), np.array([movieid])])[0][0]

def predict(answer_path, model, test_data_path):
    if os.path.isfile(answer_path):
        os.remove(answer_path)
    test_users, test_movies = read_test_data(test_data_path)
    pred = []
    for i in range(len(test_users)):
        pred.append(predict_rating(test_users[i], test_movies[i]))
    with open(answer_path, 'a') as answer:
        answer.write('TestDataID,Rating\r\n')
        index = 1
        for ele in pred:
            answer.write(str(index) +','+ str(ele) +'\r\n')
            index += 1
    return pred
    
pred = predict(answer_path, model, test_data_path)

with open(answer_path, 'a') as answer:
    answer.write('TestDataID,Rating\r\n')
    index = 1
    for ele in pred:
        answer.write(str(index) +','+ str(ele) +'\r\n')
        index += 1

In [117]:
def save_model(model):
    json_string = model.to_json()
    with open('/home/csdai0324/hw5_data/model_json.txt', 'w') as outfile:  
        json.dump(json_string, outfile)
    model.save_weights('/home/csdai0324/hw5_data/model.h5')

save_model(model)

In [120]:
model_json_path = '/home/csdai0324/hw5_data/model_json.txt'
model_weigth_path = '/home/csdai0324/hw5_data/model.h5'
def load_model(model_json_path, model_weight_path):
    with open(model_json_path) as json_file:  
        model_json = json.load(json_file)
    model_ = model_from_json(model_json)
    model_.load_weights(model_weight_path)
    return model_
model1 = load_model(model_json_path, model_weigth_path)
pred = predict(answer_path, model1, test_data_path)

/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:1252: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)
